In [ ]:
# In this notebook, we will use stats of soccer players on the 2019 FIFA roster to predict their value and wage.

In [ ]:
# Load data from csv file.

In [1]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': '3SRWJdZAZuTWsEkdcy-zVHrwCLnm4zfxZNtU1JfeoaBe',
    'service_id': 'iam-ServiceId-a7d6e3b0-c39e-4bfa-b544-cd1ca735bd64',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token'}

configuration_name = 'os_cef4952e05d8448491fd616525870fe7_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('data.csv', 'default-donotdelete-pr-veoveo1lavfexy'))
df_data_1.show()


+---+------+-----------------+---+--------------------+-----------+--------------------+-------+---------+-------------------+--------------------+-------+-----+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|_c0|    ID|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|               Club|           Club Log

In [ ]:
# Get rid of spaces in column names.

In [2]:
def nospace(name):
    name = name.split(" ")
    name = "_".join(name)
    return name

In [3]:
new_columns = list(map(nospace,df_data_1.columns))
old_columns = df_data_1.columns
replace_list = ["`" + old_c + "`" + " AS " + new_c for old_c, new_c in zip(old_columns,new_columns) ]
replace_str = ", ".join(replace_list)
replace_str = "SELECT " + replace_str + " FROM data1"
df_data_1.createOrReplaceTempView('data1')
df_data_1  = spark.sql(replace_str)

In [ ]:
# Upload dataframe to .parquet file on cloud obect store.

In [4]:
df_data_1 = df_data_1.repartition(1)
df_data_1.write.parquet(cos.url('FIFA.parquet', 'default-donotdelete-pr-veoveo1lavfexy'),mode = 'overwrite')


In [ ]:
# Read data back into dataframe df

In [5]:
df = spark.read.parquet(cos.url('FIFA.parquet', 'default-donotdelete-pr-veoveo1lavfexy'))

In [ ]:
# FEATURE ENGINEERING:

In [ ]:
#Remove unecessary features.

In [6]:
drop_list = ['Photo','Flag',"Club_Logo","Special","Real_Face","Loaned_From","ID",'LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM','CM',
             'RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB','Body_Type','Joined',
            'Contract_Valid_Until','Club','Jersey_Number']
for col in drop_list:
    df = df.drop(col)
df.createOrReplaceTempView('FIFA')

In [ ]:
# Replace Nationality feature with Region.

In [7]:

df_countries = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('countries of the world.csv', 'default-donotdelete-pr-veoveo1lavfexy'))
scont = pyspark.sql.SQLContext(spark.sparkContext)

def no_end_space(s):
    while s[-1]== ' ':
        s= s[:-1]
    return s

scont.registerFunction("no_end_space",no_end_space)

df_countries.createOrReplaceTempView('countries')
df_countries = spark.sql("Select no_end_space(Country) AS Country, no_end_space(Region) AS Region FROM countries")
df_countries.createOrReplaceTempView('countries')
df = spark.sql("select * FROM FIFA LEFT JOIN countries ON FIFA.Nationality = countries.Country")
df.createOrReplaceTempView('FIFA')
df = df.drop('Nationality').drop('Country')
df.createOrReplaceTempView('FIFA')

In [ ]:
# Get rid of rows without a valid value for Value or Wage. These are the variables we will try to predict.
# Set missing rows in Release_Clause column to zero.

In [8]:
def check_last(s):
    if s[-1]=='M' or s[-1]=='K':
        return True
    else:
        return False
    
def check_last_release(s):
    if s==None:
        return '0'
    if s[-1]!='M' and s[-1]!='K':
        return '0'
    return s    

scont.registerFunction("check_last",check_last)
scont.registerFunction("check_last_release",check_last_release)

df = spark.sql("select * FROM FIFA WHERE CAST(check_last(Value) as boolean) and CAST(check_last(Wage) as boolean)")
df.createOrReplaceTempView('FIFA')

def get_value(s):
    
    if s== '0':
        return s
    last = s[-1]
    s = s[1:-1]
    if "." not in s:
        if last == 'M':
            s = s + '000000'
        elif last == 'K':
            s = s + '000'
    else:
        s_li = s.split(".")
        ss = s_li[1]
        if last == 'M':
            mag = 6
        elif last == 'K':
            mag =3
        if len(ss)<mag:
            rem = mag - len(ss)
            for j in range(rem):
                ss = ss + '0'
        elif len(ss)>mag:
            ssf = ss[0:mag]
            sss = ss[mag:]
            ss = ssf + "." + sss
        
        s_li[1] = ss
        s = "".join(s_li)
    return s

scont.registerFunction("get_value",get_value)

value_df = spark.sql('''
select _c0 as _c0_c, CAST(get_value(Value) as double) as Value_float, 
CAST(get_value(Wage) as double) as Wage_float,
CAST(get_value(check_last_release(Release_Clause)) as double) as Release_Clause_float from FIFA
''')
value_df.createOrReplaceTempView('values')
df = spark.sql("select * FROM FIFA JOIN values on FIFA._c0 = values._c0_c")
df = df.drop('_c0_c').drop('Value').drop('Wage').drop('Release_Clause')
df = df.withColumnRenamed('Value_float', 'Value')
df = df.withColumnRenamed('Wage_float', 'Wage')
df = df.withColumnRenamed('Release_Clause_float', 'Release_Clause')
df.createOrReplaceTempView('FIFA')


In [ ]:
# Split work_rate column into two columns. Encode both work_rates as an integer.

In [9]:
def wrate(s,pos):
    if s is None:
        return None
    s_li = s.split("/")
    if pos == 'f':
        el = s_li[0]
    else:
        el = s_li[1][1:]
    if el == 'Low':
        return '1'
    if el == 'Medium':
        return '2'
    if el == 'High':
        return '3'
    
scont.registerFunction("wrate",wrate)

wrate_df = spark.sql('''
select _c0 as _c0_c, CAST(wrate(Work_Rate,'f') as double) as Work_Rate_1, 
CAST(wrate(Work_Rate,'s') as double) as Work_Rate_2 from FIFA
''')

wrate_df.createOrReplaceTempView('wrate')
df = spark.sql("select * FROM FIFA JOIN wrate on FIFA._c0 = wrate._c0_c")
df = df.drop('_c0_c').drop('Work_Rate')
df.createOrReplaceTempView('FIFA')

In [ ]:
# Encode the height as a double.

In [10]:
def Height_func(s):
    if s is None:
        return None
    s_li = s.split("'")
    feet = float(s_li[0])
    inch = float(s_li[1])
    height = feet + inch/12
    return str(height)

scont.registerFunction("Height_func",Height_func)

Height_df = spark.sql("select _c0 as _c0_c, CAST(Height_func(Height) as double) as Height_float from FIFA")
Height_df.createOrReplaceTempView('Height')
df = spark.sql("select * FROM FIFA JOIN Height on FIFA._c0 = Height._c0_c")
df = df.drop('_c0_c').drop('Height')
df = df.withColumnRenamed('Height_float', 'Height')
df.createOrReplaceTempView('FIFA')

In [ ]:
# Encode the weight as a double.

In [11]:
def Weight_func(s):
    if s is None:
        return None
    return s[:-3]

scont.registerFunction("Weight_func",Weight_func)
Weight_df = spark.sql("select _c0 as _c0_c, CAST(Weight_func(Weight) as double) as Weight_float from FIFA")
Weight_df.createOrReplaceTempView('Weight')
df = spark.sql("select * FROM FIFA JOIN Weight on FIFA._c0 = Weight._c0_c")
df = df.drop('_c0_c').drop('Weight')
df = df.withColumnRenamed('Weight_float', 'Weight')
df.createOrReplaceTempView('FIFA')

In [ ]:
# Reduce the number of positions to either Goalie, Back, Midfielder, Winger, or Striker.

In [12]:
def Pos_func(s):
    if s is None:
        return None
    if 'G' in s:
        return 'G'
    if 'B' in s:
        return 'B'
    if 'M' in s:
        return 'M'
    if 'W' in s:
        return 'W'
    if 'S' in s or 'F' in s:
        return 'S'
    
scont.registerFunction("Pos_func",Pos_func)
Pos_df = spark.sql("select _c0 as _c0_c, Pos_func(Position) as Posit from FIFA")
Pos_df.createOrReplaceTempView('Pos')
df = spark.sql("select * FROM FIFA JOIN Pos on FIFA._c0 = Pos._c0_c")
df = df.drop('_c0_c').drop('Position')
df = df.withColumnRenamed('Posit', 'Position')
df.createOrReplaceTempView('FIFA')

In [ ]:
# Transform all numerical columns from strings to doubles.

In [13]:
cols_cast = df.columns
cols_non_num = ['Name','Preferred_Foot','Position','Region']
for col in cols_non_num:
    cols_cast.remove(col)
    
sel_str  = "select "
for col in cols_non_num:
    sel_str = sel_str + col + ", "
for col in cols_cast:
    sel_str = sel_str + "CAST(" + col +  " as double) as " + col + ", "
sel_str = sel_str[:-2]
sel_str = sel_str + " from FIFA"

df = spark.sql(sel_str)
df.createOrReplaceTempView('FIFA')

In [ ]:
# Average the best num_skills skills for each player.

In [14]:
cols = df.columns
skills = cols[11:45]
other_cols = cols[:11]
other_cols.extend(cols[45:])

num_skills=5
def avg_best(*scores):
    scores = list(scores)
    while None in scores:
        scores.remove(None)
    scores.sort()
    scores.reverse()
    scores = scores[0:num_skills]
    return sum(scores)/num_skills

scont.registerFunction("avg_best",avg_best)

sel_str = "select "
for col in other_cols:
    sel_str = sel_str + col + ", "
sel_str = sel_str + "CAST(avg_best("
for col in skills:
    sel_str = sel_str + col +", "
sel_str = sel_str[:-2]
sel_str = sel_str  + ") as double) as avg_best_skills from FIFA"
sel_str

df = spark.sql(sel_str)

df.createOrReplaceTempView('FIFA')

In [ ]:
# Get rid of rows where we don't know the player's position or what region they are from.

In [15]:
df = spark.sql("select * from FIFA where Position is not NULL  AND Region is not NULL")
df.createOrReplaceTempView('FIFA')

In [ ]:
# Replace all missing values in numerical columns with their average accross the dataset.

In [16]:
cols = df.columns
flt_cols = cols[5:]

sel_str = "select "
for col in flt_cols:
    sel_str = sel_str + 'ROUND(AVG(' + col + ')) as ' + col + ', '
sel_str = sel_str[: -2]
sel_str = sel_str + ' from FIFA'
aves =spark.sql(sel_str).collect()

aves_dict = {}
for col in flt_cols:
    aves_dict[col] = aves[0].asDict()[col]
    
df = df.fillna(aves_dict)
df.createOrReplaceTempView('FIFA')

In [ ]:
# Check to make sure that columns have no missing values.

In [17]:
sel_str = "select "
for col in df.columns:
    sel_str = sel_str + 'Count(' + col + ') as ' + col + ', '
sel_str = sel_str[: -2]
sel_str = sel_str + ' from FIFA'
spark.sql(sel_str).collect()

[Row(Name=17905, Preferred_Foot=17905, Position=17905, Region=17905, _c0=17905, Age=17905, Overall=17905, Potential=17905, International_Reputation=17905, Weak_Foot=17905, Skill_Moves=17905, Value=17905, Wage=17905, Release_Clause=17905, Work_Rate_1=17905, Work_Rate_2=17905, Height=17905, Weight=17905, avg_best_skills=17905)]

In [ ]:
# Convert all categorical columns with strings to integers indexing the different categories.

In [19]:
from pyspark.ml.feature import StringIndexer

ind_columns = ['Preferred_Foot','Position','Region']

for col in ind_columns:
    indexer = StringIndexer(inputCol = col, outputCol = col + '_0')
    df = indexer.fit(df).transform(df)

for col in ind_columns:
    df = df.drop(col)
    df = df.withColumnRenamed(col + '_0', col)
    
df.createOrReplaceTempView('FIFA')

In [ ]:
# Split numerical variables into bins to be used as categorical variables.

In [20]:
quantile_cols = ['Age','Overall','Potential','Height','Weight','avg_best_skills']
other_cols = df.columns
for col in quantile_cols:
    other_cols.remove(col)
    
stat = pyspark.sql.DataFrameStatFunctions(df)
quantile_dict = {}
for indx,col in enumerate(quantile_cols):
    quantile_dict[indx] = stat.approxQuantile(col,[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,.95,.99,.999],0)
    
def find_indx(value,indx):
    quantiles = quantile_dict[indx]
    if value >= quantiles[-1]:
        return len(quantiles)
    new_q = list(map(lambda x: x>=value,quantiles))
    ind = new_q.index(True)
    return ind

scont.registerFunction("find_indx",find_indx)

sel_str = "select "
for col in other_cols:
    sel_str = sel_str + col + ", "
for indx,col in enumerate(quantile_cols):
    sel_str = sel_str + "CAST(find_indx(" + col + "," + str(indx) + ") as double) as " + col + ", "
sel_str = sel_str[:-2]
sel_str = sel_str + ' from FIFA'

df = spark.sql(sel_str)
df.createOrReplaceTempView('FIFA')

In [ ]:
# Create features column containing vector of features.

In [21]:
feature_cols = df.columns
for col in ['Name','_c0','Value','Wage','Release_Clause']:
    feature_cols.remove(col)
    
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
VA = VectorAssembler(inputCols = feature_cols, outputCol = 'features')
df = VA.transform(df)

df.createOrReplaceTempView('FIFA')

In [ ]:
# Split the data into train_set, val_set, and test_test. Fit Random Forest regression algorithm to train data 
# to predict the value of each player.

In [23]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
(train_df,val_df, test_df) = df.randomSplit([0.9,0.05, 0.05])
rf = RandomForestRegressor( labelCol = 'Value',maxDepth=10, numTrees=40)
model = rf.fit(train_df)

In [ ]:
# Evaluate model on train set.

In [24]:
predictions = model.transform(train_df)
predictions.select("prediction", "Value", "features").show(20)

+--------------------+---------+--------------------+
|          prediction|    Value|            features|
+--------------------+---------+--------------------+
|     868366.72187714| 825000.0|[1.0,2.0,2.0,1.0,...|
|   2260826.533117898|1900000.0|[1.0,3.0,4.0,2.0,...|
|   5250792.988781194|6000000.0|[1.0,3.0,3.0,2.0,...|
|  407123.50770166266| 375000.0|[1.0,2.0,2.0,3.0,...|
|   6912280.787282777|7500000.0|[2.0,3.0,3.0,2.0,...|
|1.1232731638615882E7|   1.15E7|[1.0,2.0,4.0,3.0,...|
|1.3202053648634627E7|    1.3E7|[3.0,4.0,4.0,3.0,...|
|  161570.65284446787| 170000.0|[1.0,2.0,1.0,2.0,...|
|   263240.7062658176| 230000.0|[1.0,2.0,2.0,2.0,...|
|  170633.42212346237| 200000.0|[1.0,2.0,2.0,2.0,...|
|  1594865.1727441791|1200000.0|[1.0,3.0,3.0,2.0,...|
|   4742173.385044406|4700000.0|[2.0,3.0,3.0,3.0,...|
|   95484.96994103766|  80000.0|[1.0,2.0,2.0,2.0,...|
|   4560840.370777808|4400000.0|[1.0,4.0,3.0,2.0,...|
|  2953426.8921676036|2200000.0|[2.0,4.0,3.0,3.0,...|
|  377563.42831476167| 32500

In [ ]:
# Calculate rmse on train set.

In [25]:
evaluator = RegressionEvaluator(labelCol="Value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
rmse

675225.2278570898

In [ ]:
# Evaluate model on validation set.

In [26]:
predictions = model.transform(val_df)
predictions.select("prediction", "Value", "features").show(20)


+--------------------+---------+--------------------+
|          prediction|    Value|            features|
+--------------------+---------+--------------------+
|  1543503.1218146775|1400000.0|[1.0,2.0,1.0,2.0,...|
|   303351.1443392128| 375000.0|[1.0,3.0,2.0,2.0,...|
|1.1392524876373928E7|   1.15E7|[3.0,2.0,2.0,2.0,...|
|1.8710766210645325E7|    1.4E7|[2.0,3.0,4.0,3.0,...|
|   6136237.840264907|5000000.0|[1.0,3.0,4.0,2.0,...|
|   1618296.285566496|1900000.0|[1.0,3.0,5.0,3.0,...|
|  126880.15396327137| 150000.0|[1.0,3.0,2.0,1.0,...|
|  2965542.3322684728|3000000.0|[1.0,3.0,4.0,2.0,...|
|   957306.2928896623| 900000.0|[1.0,3.0,3.0,3.0,...|
|   380968.5685684047| 180000.0|[1.0,4.0,2.0,3.0,...|
|   213669.5139796901| 180000.0|[1.0,3.0,2.0,2.0,...|
|   1445375.729445854|1600000.0|[1.0,3.0,3.0,3.0,...|
|  349557.35659558506| 400000.0|[1.0,4.0,2.0,2.0,...|
|   4125806.611115633|4200000.0|[1.0,2.0,3.0,3.0,...|
|   387980.7340639687| 450000.0|[1.0,3.0,2.0,2.0,...|
|   973897.5101068948|110000

In [ ]:
# Calculate rmse on validation set. 

In [29]:
evaluator = RegressionEvaluator(labelCol="Value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
rmse

1022283.8831797091

In [ ]:
# Evaluate model on test set.

In [30]:
predictions = model.transform(test_df)
predictions.select("prediction", "Value", "features").show(20)


+--------------------+---------+--------------------+
|          prediction|    Value|            features|
+--------------------+---------+--------------------+
|   558442.2992566231| 525000.0|[1.0,3.0,2.0,1.0,...|
|    537916.497736715| 425000.0|[1.0,3.0,3.0,3.0,...|
|   725265.2094536925| 750000.0|[1.0,2.0,3.0,2.0,...|
|   264950.2957194938| 300000.0|[1.0,3.0,2.0,2.0,...|
|   267311.4924614284| 200000.0|[1.0,2.0,2.0,2.0,...|
|  433089.19497975864| 400000.0|[1.0,3.0,2.0,2.0,...|
|  1343440.4115827894|1500000.0|[1.0,3.0,2.0,2.0,...|
|   6208119.207451368|6000000.0|[2.0,3.0,2.0,2.0,...|
|   643735.8629622166| 950000.0|[1.0,4.0,2.0,2.0,...|
|   295097.6208594922| 325000.0|[1.0,3.0,2.0,2.0,...|
|  3593717.9490807517|3500000.0|[1.0,3.0,2.0,2.0,...|
|   2175394.736218237|2700000.0|[1.0,2.0,1.0,2.0,...|
|    6685905.14510939|7500000.0|[2.0,3.0,3.0,3.0,...|
|   399704.5867146278| 325000.0|[1.0,3.0,1.0,2.0,...|
|  1052020.6305078263|1200000.0|[1.0,2.0,3.0,3.0,...|
|  434585.70611710846| 37500

In [ ]:
# Calculate rmse on test set. It is less than a million Euros. Hence, our algorithm is pretty good.

In [31]:
evaluator = RegressionEvaluator(labelCol="Value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
rmse

875765.0537698619